<a href="https://colab.research.google.com/github/PremaKathiresanVasagam/TSAI---EVA5/blob/master/Session5_CodeDrillDown/Session5_Model3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(6),
            nn.Conv2d(6, 6, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(6),
            nn.Conv2d(6, 8, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.15)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 10, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Conv2d(10, 10, 3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            #nn.MaxPool2d(2, 2),
            nn.Conv2d(10, 10, 3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Conv2d(10, 12, 3, stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(0.15)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(12, 18, 3),
            #nn.ReLU(),
            nn.Conv2d(18, 10, 3),
            nn.AvgPool2d(3)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [18]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]              60
              ReLU-2            [-1, 6, 28, 28]               0
       BatchNorm2d-3            [-1, 6, 28, 28]              12
            Conv2d-4            [-1, 6, 28, 28]             330
              ReLU-5            [-1, 6, 28, 28]               0
       BatchNorm2d-6            [-1, 6, 28, 28]              12
            Conv2d-7            [-1, 8, 28, 28]             440
              ReLU-8            [-1, 8, 28, 28]               0
       BatchNorm2d-9            [-1, 8, 28, 28]              16
        MaxPool2d-10            [-1, 8, 14, 14]               0
          Dropout-11            [-1, 8, 14, 14]               0
           Conv2d-12           [-1, 10, 14, 14]             730
             ReLU-13           [-1, 10, 14, 14]               0
      BatchNorm2d-14           [-1, 10,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [19]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [20]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [21]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Loss=0.035096924751996994 Batch_id=468 Accuracy=90.58: 100%|██████████| 469/469 [00:29<00:00, 15.90it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0610, Accuracy: 9807/10000 (98.07%)



Loss=0.10661861300468445 Batch_id=468 Accuracy=97.56: 100%|██████████| 469/469 [00:28<00:00, 16.17it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0665, Accuracy: 9789/10000 (97.89%)



Loss=0.06670858711004257 Batch_id=468 Accuracy=98.15: 100%|██████████| 469/469 [00:29<00:00, 16.14it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0208, Accuracy: 9936/10000 (99.36%)



Loss=0.08355492353439331 Batch_id=468 Accuracy=98.97: 100%|██████████| 469/469 [00:28<00:00, 16.32it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0212, Accuracy: 9931/10000 (99.31%)



Loss=0.03398117050528526 Batch_id=468 Accuracy=99.05: 100%|██████████| 469/469 [00:29<00:00, 15.93it/s]



Test set: Average loss: 0.0207, Accuracy: 9932/10000 (99.32%)



**Target:**

Build a base model with 
* <15k Parameters and <15 Epochs


**Results:**

Number of convolutional layers: 3

Parameters: 10866

Epochs = 15

Train Accuracy: 99.19%

Test Accuracy: 99.40%

**Analysis:**

* Included BatchNorm and dropout - Learning is faster and able to attain accuracy >99.4% after the few initial layers. Overfittig didn't occure here.

* Having brought down the channel size in the convolutional layers has reduced the number of parameters from 13k to 11k.


